In [ ]:
# import libraries 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [ ]:
#define AttributeError exception for get_text

def txt_ex(i,select,class_):
    try:
        return i.find(select,class_).get_text().strip()
    except AttributeError as Err:
        return None

In [ ]:
#define AttributeError exception for links
def link_ex(i):
    try:
        return i.get('href')
    except AttributeError as Err:
        return None

In [ ]:
#define AttributeError exception for find_alls
def finds_all(i,select,class_):
    try:
        return i.find_all(select,class_)
    except AttributeError as Err:
        return None

In [ ]:
#define pulling data from website

def pull_data(URL):

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page=requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    
    return soup2

In [ ]:
urls=[] 

#Scrape pages 1 to 20 of the website

for i in range (1,21):
    
    URL=f'https://www.etsy.com/c/books-movies-and-music/books/childrens-books?ref=pagination&page={i}' # URL address for children book at etsy 
    
    soup=pull_data(URL) #pull html data of the url
    ols=soup.find("ol")
    lis=ols.find_all('li')
    
    for link in lis:
        
        # Extract title of the product
        title=txt_ex(link,'h3',"wt-text-caption v2-listing-card__title wt-text-truncate")
        
        # Extract the links of product page
        a_s=link.find('a')
        links=link_ex(a_s) 
        
        # pull html data of individual product page
        soup=pull_data(links) 
        
        #Extract the price of the product with removing new line to convert it to a single line
        price=txt_ex(soup, 'p',"wt-text-title-03 wt-mr-xs-1") 
        char_to_remove= "\n  "
        price = price.replace(char_to_remove, ",")
        
        #Extract Shop name of the product
        shop=txt_ex(soup,'span', "wt-text-title-small")
        
        # Extract the overall_star of the product
        overall_star=txt_ex(soup, 'span', "wt-display-inline-block wt-mr-xs-1")
        
        # Extract the individual star rating, reviewe, date of the review 
        divs=soup.find('div', class_="wt-pl-xs-0 wt-mb-xs-2 wt-mb-lg-6")
        divs2=finds_all(divs,'div',"wt-grid__item-xs-12")
        if divs2 is not None:
            for reveiw in divs2:
                star=txt_ex(reveiw,'span', "wt-screen-reader-only")
                reviews=txt_ex(reveiw,'p',"wt-text-truncate--multi-line")
                date=txt_ex(reveiw,'p',"wt-text-caption wt-text-gray")
                char_to_remove= "\n  "
                date = date.replace(char_to_remove, ",")
                urls.append([title, price, shop, overall_star, star, reviews, date, links]) # Append all datain a list 


In [ ]:
# Create CSV file and emport all data in the file
df=pd.DataFrame(urls, columns=['Title', 'Price', 'Shop_name', 'Overall_star', 'Star_rating', 'Review', 'Customer_date', 'Product_links'])
df.to_csv('Etsy_Reveiw_Data_2.csv')

In [ ]:
# Read the CSV file
df=pd.read_csv("C:/Users/ashpi/Etsy_Reveiw_Data_2.csv")
df.head()